# Infer ENDOPLASMIC RETICULUM - part 8️⃣

--------------

## OBJECTIVE:  ✅ Infer sub-cellular component ENDOPLASMIC RETICULUM (ER) in order to understand interactome 



Dependencies:
The ER  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [ ]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice, edge_preserving_smoothing_3d )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles.nuclei import infer_NUCLEI
from infer_subc_2d.organelles.soma import infer_SOMA
from infer_subc_2d.organelles.cytosol import infer_CYTOSOL
from infer_subc_2d.constants import TEST_IMG_N

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N

# IMAGE PROCESSING Objective 8:  infer ER
> Back to  [OUTLINE: Objective #5](00_pipeline_setup.ipynb#summary-of-objectives)

## summary of steps (Workflow #1 & #2)

INPUT
- channel  6
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
-  segment objects

- POST-PROCESSING
  - n/a

OUTPUT
- object PEROXISOMES 



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [ ]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [ ]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [ ]:
load_Z_from_params = False


if load_Z_from_params:

    default_params = load_parameters( test_img_name.split("/")[-1], data_root_path / "intermediate" )

    ch_to_agg = default_params["ch_to_agg"]
    nuc_ch = default_params['nuc_ch']
    optimal_Z = default_params["optimal_Z"] #find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
else:
    ch_to_agg = (1,2,3,4,5,6)
    nuc_ch = 0
    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    default_params = defaultdict(str, **{
        #"intensity_norm_param" : [0.5, 15]
        "intensity_norm_param" : [0],
        "gaussian_smoothing_sigma" : 1.34,
        "gaussian_smoothing_truncate_range" : 3.0,
        "dot_2d_sigma" : 2,
        "dot_2d_sigma_extra" : 1,
        "dot_2d_cutoff" : 0.025,
        "min_area" : 10,
        "low_level_min_size" :  100,
        "median_filter_size" : 4,
        "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
        "nuc_ch" : 0,
        "optimal_Z": optimal_Z,
    })
    save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


## get the inferred nuclei object

(takes < 1 sec)

In [ ]:


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


## get the inferred soma object

(takes < 1 sec)

In [ ]:
raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, default_params) 
CY_object =  infer_CYTOSOL(SO_object, NU_object) 


# WORKFLOW #1 

Generally following the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.

>sec61beta  is  good start for ER  as per from [Allen Cell](https://www.allencell.org/cell-observations/category/endoplasmic-reticulum).    

using [seg_sec61b.py](https://github.com/AllenInstitute/aics-segmentation/blob/master/aics-segmentation/aicssegmentation/structure_wrapper/seg_sec61b.py)
[seg_sec61b_dual.py](https://github.com/AllenInstitute/aics-segmentation/blob/master/aicssegmentation/structure_wrapper/seg_sec61b_dual.py)

[playground_Sec61b.ipynb](https://github.com/AllenInstitute/aics-segmentation/blob/master/lookup_table_demo/playground_Sec61b.ipynb)



## summary of steps


INPUT
- ch 4
- CY mask

PRE-PROCESSING
- edge preserving scaling

CORE-PROCESSING
- vesselness slice-by-slice

POST-PROCESSING
  - S  - remove objects less than 2x2 pixels (area = 4)

OUTPUT
- object PEROXISOMES 





### INPUT

In [ ]:

###################
# INPUT
###################
raw_er   = img_2D[5].copy()

# mask object
masked_er = apply_mask(raw_er, CY_object)


### PRE-PROCESSING

In [ ]:
###################
# PRE_PROCESSING
###################


intensity_norm_param = [0]  # CHECK THIS
# Linear-ish smoothing
struct_img = intensity_normalization( masked_er ,  scaling_param=intensity_norm_param)

# edge-preserving smoothing (Option 2, used for Sec61B)
structure_img_smooth = edge_preserving_smoothing_3d(struct_img)
# 10 seconds!!!!  tooooo slow... for maybe no good reason

### CORE PROCESSING

In [ ]:
###################
# CORE_PROCESSING
###################

################################
## PARAMETERS for this step ##
f2_param = [[1, 0.15]]
################################

bw = filament_2d_wrapper(structure_img_smooth, f2_param)

### POST-PROCESSING

In [ ]:

###################
# POST_PROCESSING
###################
################################
## PARAMETERS for this step ##
small_object_max = 4
################################
GO_object = size_filter_2D(bw, 
                                                min_size= small_object_max**2, 
                                                connectivity=1)
# ################################
# ## PARAMETERS for this step ## from seg_sec61b.py
# min_area = 15
# ################################
# bw = remove_small_objects(bw > 0, min_size=min_area, connectivity=1, in_place=False)
# # prune slice by slice
# for zz in range(bw.shape[0]):
#     bw[zz, :, :] = remove_small_objects(bw[zz, :, :], min_size=3, connectivity=1, in_place=False)

# ER_object = remove_small_objects(bw > 0, min_size=min_area, connectivity=1, in_place=False)



#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:

viewer = napari.view_image(
    bw,
    scale=scale
)


viewer.scale_bar.visible = True
viewer.add_image(
    structure_img_smooth,
    scale=scale
)

viewer.add_image(
    GO_object,
    scale=scale
)

### DEFINE `infer_ENDOPLASMIC_RETICULUM` function

In [ ]:
##########################
#  infer_ENDOPLASMIC_RETICULUM
##########################
def _infer_ENDOPLASMIC_RETICULUM(struct_img, CY_object,  in_params) -> tuple:
    """
    Procedure to infer ER  from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 2d image containing the ER signal

    CY_object: np.ndarray boolean
        a 2d (3D with 1 Z) image containing the cytosol mask
    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of ER
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()
    struct_img = apply_mask(struct_img,CY_object)

    ###################
    # PRE_PROCESSING
    ###################                         
    intensity_norm_param = [0]  # CHECK THIS

    struct_img = intensity_normalization(struct_img, scaling_param=intensity_norm_param)
    out_p["intensity_norm_param"] = intensity_norm_param

    # edge-preserving smoothing (Option 2, used for Sec61B)
    structure_img_smooth = edge_preserving_smoothing_3d(struct_img)


   ###################
    # CORE_PROCESSING
    ###################
    ################################
    ## PARAMETERS for this step ##
    f2_param = [[1, 0.15]]
    ################################

    struct_obj = filament_2d_wrapper(struct_img, f2_param)
    out_p["f2_param"] = f2_param 

    ###################
    # POST_PROCESSING
    ###################
 
    ################################
    ## PARAMETERS for this step ##
    small_object_max = 2 
    ################################
    # struct_obj = remove_small_objects(struct_obj>0, min_size=min_area, connectivity=1, in_place=False)
    # out_p["min_area"] = min_area 

    struct_obj = size_filter_2D(struct_obj, 
                                                    min_size= small_object_max**2, 
                                                    connectivity=1)
    out_p['small_object_max'] = small_object_max

    retval = (struct_obj,  out_p)
    return retval


# TEST  `_infer_ENDOPLASMIC_RETICULUM` function

In [ ]:

###################
# INPUT
###################
raw_er  = img_2D[5].copy()

ER_object, out_p =  _infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, default_params) 

In [ ]:

viewer.add_image(
    ER_object,
    scale=scale
)
viewer.add_labels(
    label(ER_object),
    scale=scale
)

In [ ]:
from infer_subc_2d.organelles.er import infer_ENDOPLASMIC_RETICULUM


img_2D = img_data[:,[optimal_Z],:,:].copy()
raw_er  = img_2D[5].copy()

ER_object, out_p =  infer_ENDOPLASMIC_RETICULUM(raw_er, CY_object, default_params) 

In [ ]:
# save updated parameters for ongoing testing
save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )

---------------------

 🚧 WIP 🚧 (🚨🚨🚨🚨 )

# WORKFLOW #2 (WIP)
as per 6/22 CellProfiler pipeline from MCZ
 
## summary of steps

INPUT
- ch 6
- CY mask

PRE-PROCESSING
- rescale 
- median window:4
- non-local noise reduction
  - size:5, distance:1, cut-off:0.1
- identify "TUBES"
  - enhance neurites
    - > Neurites: Neurites are taken to be long, thin features of enhanced intensity. Choose this option to enhance the intensity of the neurites using the Line structures or Tubeness methods described in a later setting.  
    - method: tubeness
    - > Tubeness: This method is an adaptation of the method used by the ImageJ Tubeness plugin. The image is smoothed with a Gaussian. The Hessian is then computed at every point to measure the intensity gradient and the eigenvalues of the Hessian are computed to determine the magnitude of the intensity. The absolute maximum of the two eigenvalues gives a measure of the ratio of the intensity of the gradient in the direction of its most rapid descent versus in the orthogonal direction. The output image is the absolute magnitude of the highest eigenvalue if that eigenvalue is negative (white neurite on dark background), otherwise, zero. 
    - smoothing: 2 pix

CORE-PROCESSING
- identify primary objects "TUBES"
    - adaptive Sauvola
      - threshold smoothing scale: 0
      - threshold correction factor: .6
      - threshold bounds: (0. ,1.0)
      - adaptive window: 20 pixels
- identify primary objects "SHEETS"
  - ER masked w/ CY
  - adaptive Otsu
      - diameter: (2,50)
    - two classes
      - threshold smoothing scale: 0
      - threshold correction factor: 1
      - threshold bounds: (0.18, .25)
      - adaptive window: 10 pixels
- combine "TUBES" and "SHEETS"



OUTPUT
- object ER




### INPUT

In [ ]:

###################
# INPUT
###################
struct_img_raw = img_data[5,:,:,:].copy()

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =2  

gaussian_smoothing_sigma = 10
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]


### PRE-PROCESSING

In [ ]:
###################
# PRE_PROCESSING
###################

intensity_norm_param = [0] # 

# Linear-ish smoothing
raw_mito = intensity_normalization( struct_img_raw ,  scaling_param=intensity_norm_param)

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

struct_img = median_filter(raw_mito,    size=med_filter_size  )

structure_img_smooth = image_smoothing_gaussian_3d(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

### CORE PROCESSING

In [ ]:
# enhance spreckles - 40
big_struct_rad = 40
big_img = _enhance_speckles(struct_img.copy(),big_struct_rad, True)



In [ ]:

# enhance spreckles - 10
sm_struct_rad = 20
sm_img = _enhance_speckles(struct_img.copy(),sm_struct_rad, True)


#adaptive_otsu(big_struct) # three class - middle foreground
# adaptive window- 20
#size: 10,100
# threshold smooth 1.34
# threshold correction 1
# threshold (0.1497,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 10
bw_big, _bw_low_level = MO(big_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)

#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold


# enhance speckles
#   adaptive_sauvola(sm_struct) 
# adaptive window- 10
#size: 2,10
# threshold smooth 1.34
# threshold correction 1
# threshold (0.05,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 2
bw_sm, _bw_low_level = MO(sm_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)
#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold



### POST-PROCESSING

In [ ]:
###################
# POST_PROCESSING
###################


# 3D
# cleaned_img = remove_small_objects(removed_holes>0, 
#                                                             min_size=minArea, 
#                                                             connectivity=1, 
#                                                             in_place=False)
small_object_max = 10
cleaned_img_big = size_filter(bw_big, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)


#                                                             in_place=False)
small_object_max = 2
cleaned_img_sm = size_filter(bw_sm, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)



cleaned_img = np.logical_or(cleaned_img_big, cleaned_img_sm)



#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )
else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )

viewer.scale_bar.visible = True
viewer.add_image(
    cleaned_img_big,
    scale=scale
)

viewer.add_image(
    big_img,
    scale=scale
)
viewer.add_image(
    cleaned_img_big,
    scale=scale
)
viewer.add_image(
    sm_img,
    scale=scale
)

# TEST  `infer_ENDOPLASMIC_RETICULUM` function